# Step 2: Analytical Robustness

This notebook demonstrates how to derive origin–destination (OD) flows from the synthetic CDR datasets and assess their robustness to reasonable method choices. Following the exercise instructions, we compare OD matrices obtained under two different speed thresholds and quantify how sensitive the flows are to the movement definition.

## 1 – Setup and Data Loading
We start by importing the necessary libraries and loading the input datasets. This example uses `pandas` for tabular data, `geopandas` for spatial joins, and optionally Plotly for interactive maps.

We load:
* The synthetic CDR events (`events.csv`), diaries (`diaries.csv`) and cell tower information (`cells.csv`).
* Population data and administrative boundaries (`BRA_adm2_pop.csv` and `BRA_adm2.shp`).

These datasets will enable us to map user home locations to districts, compute OD flows, and compare the results of different methods.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Attempt to import geopandas and plotly; if unavailable, raise a user‑friendly message
try:
    import geopandas as gpd
except ImportError as e:
    raise ImportError("geopandas is required for spatial operations. Install it via pip (pip install geopandas).")

try:
    import plotly.express as px
except ImportError as e:
    raise ImportError("plotly is required for interactive maps. Install it via pip (pip install plotly).")


# File paths (adjust if you organise your files differently)
events_fp = 'https://github.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/raw/refs/heads/main/2.5.1_input_data/events.parquet'
diaries_fp = 'https://github.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/raw/refs/heads/main/2.5.1_input_data/diaries.csv'
cells_fp = 'https://github.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/raw/refs/heads/main/2.5.1_input_data/cells.csv'
adm2_pop_fp = 'https://github.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/raw/refs/heads/main/2.5.1_input_data/BRA_adm2_pop.csv'

# Load CSV data
events = pd.read_parquet(events_fp)
diaries = pd.read_csv(diaries_fp)
cells = pd.read_csv(cells_fp)
adm2_pop = pd.read_csv(adm2_pop_fp)

# Load only the Admin2 boundary from the shapefile zip using geopandas
# The shapefile zip contains BRA_adm0.shp, BRA_adm1.shp, BRA_adm2.shp, BRA_adm3.shp.
adm2_gdf = gpd.read_file(f"https://github.com/Flowminder/WB-GDF-Modern-Data-Workflows-Code-Tasks/raw/refs/heads/main/2.5.1_input_data/BRA_adm2.gpkg")
# Ensure CRS is WGS84
adm2_gdf = adm2_gdf.to_crs(epsg=4326)

print(f'Loaded {len(events):,} CDR events, {len(diaries):,} diary records, {len(cells):,} cell towers and {len(adm2_gdf):,} Admin2 polygons.')

## 2 – Map Home Locations to Admin 2
The first step is to derive a home district for each user. We select rows from `diaries` where `stay_type == 'home'`, convert the coordinates into point geometries, and perform a spatial join with the Admin 2 boundaries.

This mapping allows us to: (1) count the number of subscribers living in each district, and (2) filter to a target set of districts based on population or subscriber counts (e.g., São Paulo and surrounding areas).

In [ ]:
import json
import folium
from folium import plugins
import branca.colormap as cm

# Filter to home locations only
home_df = diaries[diaries['stay_type'] == 'home'].copy()
home_df = home_df.dropna(subset=['longitude', 'latitude'])

# Convert to GeoDataFrame (WGS84)
home_gdf = gpd.GeoDataFrame(
    home_df,
    geometry=gpd.points_from_xy(home_df['longitude'], home_df['latitude']),
    crs='EPSG:4326'
)

# Spatial join: assign each home point to an Admin2 polygon
home_join = gpd.sjoin(
    home_gdf,
    adm2_gdf[['ID_2', 'NAME_2', 'geometry']],
    how='left',
    predicate='within'
)

# Count subscribers per Admin2 (unique user_id)
home_counts = (
    home_join
    .groupby(['ID_2', 'NAME_2'])['user_id']
    .nunique()
    .reset_index(name='num_subscribers')
    .sort_values('num_subscribers', ascending=False)
)

# Merge with synthetic census population (adm2_pop has columns: ID_2, NAME_2, pop_sum)
adm2_pop_renamed = adm2_pop.rename(columns={'ID_2': 'ID_2', 'NAME_2': 'NAME_2'})
home_counts = home_counts.merge(
    adm2_pop_renamed[['ID_2', 'pop_sum']],
    on='ID_2',
    how='left'
)

# Compute coverage ratio (optional)
home_counts['coverage_ratio'] = home_counts['num_subscribers'] / home_counts['pop_sum']

# Select target Admin2 districts – e.g., top 12 by number of subscribers
target_admin2 = home_counts.head(12)['ID_2'].tolist()
print('Top 12 Admin2 districts by number of subscribers:')
display(home_counts.head(12))

# Prepare Admin2 polygons for Folium
adm2_plot = adm2_gdf[['ID_2', 'NAME_2', 'geometry']].copy()
adm2_plot['ID_2'] = adm2_plot['ID_2'].astype(int).astype(str)  # Remove .0

# Count unique users per Admin2 (not all records)
home_counts_for_map = (
    home_join.dropna(subset=['ID_2'])
    .groupby('ID_2')['user_id']
    .nunique()
    .reset_index(name='num_homes')
)
home_counts_for_map['ID_2'] = home_counts_for_map['ID_2'].astype(int).astype(str)

# Merge home counts with Admin2 polygons
adm2_plot = adm2_plot.merge(home_counts_for_map, on='ID_2', how='left')
adm2_plot['num_homes'] = adm2_plot['num_homes'].fillna(0).astype(int)

# Convert to GeoJSON for folium
adm2_plot_json = json.loads(adm2_plot.to_json())

# Get primary home location for each user (most frequent location)
home_plot = home_join.dropna(subset=['ID_2']).copy()

# For each user, find their most common home location
user_primary_homes = []
for user_id in home_plot['user_id'].unique():
    user_homes = home_plot[home_plot['user_id'] == user_id]
    # Group by lat/lon and count occurrences
    location_counts = user_homes.groupby(['latitude', 'longitude', 'ID_2', 'NAME_2']).size().reset_index(name='count')
    # Get the most frequent location
    primary_home = location_counts.loc[location_counts['count'].idxmax()]
    user_primary_homes.append({
        'user_id': user_id,
        'latitude': primary_home['latitude'],
        'longitude': primary_home['longitude'],
        'ID_2': primary_home['ID_2'],
        'NAME_2': primary_home['NAME_2']
    })

home_plot = pd.DataFrame(user_primary_homes)
home_plot['ID_2'] = home_plot['ID_2'].astype(int).astype(str)  # Remove .0

print(f"Total unique users with home locations: {len(home_plot)}")

# Create base map
m = folium.Map(
    location=[home_plot['latitude'].mean(), home_plot['longitude'].mean()],
    zoom_start=6,
    tiles='CartoDB positron'
)

# Create colormap for choropleth (only for values > 0)
max_homes = adm2_plot[adm2_plot['num_homes'] > 0]['num_homes'].max()
colormap = cm.LinearColormap(
    colors=['#FFFFCC', '#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026'],
    vmin=1,
    vmax=max_homes,
    caption='Number of Homes'
)

# Function to get fill color based on number of homes
def get_fill_color(num_homes):
    if num_homes < 15:
        return '#E0E0E0'  # Gray for 0
    else:
        return colormap(num_homes)

# Add choropleth layer
folium.GeoJson(
    adm2_plot_json,
    name='Admin2 Districts',
    style_function=lambda feature: {
        'fillColor': get_fill_color(
            adm2_plot[adm2_plot['ID_2'] == feature['properties']['ID_2']]['num_homes'].values[0]
            if len(adm2_plot[adm2_plot['ID_2'] == feature['properties']['ID_2']]) > 0 else 0
        ),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['ID_2', 'NAME_2', 'num_homes'],
        aliases=['ID:', 'District:', 'Homes:'],
        localize=True
    )
).add_to(m)

# Create a feature group for home points (can be toggled on/off)
home_layer = folium.FeatureGroup(name='Home Locations', show=False)

# Add home points
for idx, row in home_plot.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=0.5,
        color='blue',
        fill=True,
        fillColor='blue',
        fillOpacity=0.5,
        popup=f"User ID: {row['user_id']}",
        tooltip=f"User ID: {row['user_id']}"
    ).add_to(home_layer)

home_layer.add_to(m)

# Add layer control to toggle home locations on/off
folium.LayerControl().add_to(m)

# Add colormap legend
colormap.add_to(m)

m

#Optionally save the map to an HTML file
# m.save("home_location_map_15+.html")

## 3 – Reuse Continuity Model Functions

To derive OD flows from raw CDR events, we reuse functions from the **Continuity Models** notebook.  
These functions perform the following steps:

* **`add_coordinates_in_meters_to_events(events_df, cells_df)`** – merges event records with tower coordinates and projects them to a metric CRS (UTM) so that distances can be measured in metres.
* **`add_speed_metric(df)`** – computes the speed (km/h) of movement between consecutive cell changes for each user.
* **`determine_stay_move_from_speed(df, high_speed_threshold_kmh, low_speed_threshold_kmh)`** – classifies each event as `"move"` or `"stay"` based on two speed thresholds. Rows where speed > `high_speed_threshold_kmh` are marked as strong moves; contiguous rows with speed > `low_speed_threshold_kmh` are also included in the move segment.
* **`collapse_stay_move(df)`** – collapses consecutive events of the same type into segments and computes a time-weighted centroid for stay segments.  The result is a table of stay and move segments with start/end times and coordinates.

We copy these functions to here for convenience.  You can adjust the speed thresholds later to explore analytical robustness.


In [ ]:
import geopandas as gpd
import numpy as np

def add_coordinates_in_meters_to_events(events_df: pd.DataFrame, cells_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge events with cell coordinates (longitude/latitude) and project them to a metric CRS (UTM).
    Returns the events dataframe with additional columns: longitude, latitude, x, y (in metres).
    """
    merged_df = events_df.merge(cells_df, on='cell_id', how='left')
    gdf = gpd.GeoDataFrame(
        merged_df,
        geometry=gpd.points_from_xy(merged_df['longitude'], merged_df['latitude']),
        crs='EPSG:4326'
    )
    # Project to UTM zone 23S (EPSG:31983) – appropriate for Brazil
    gdf_projected = gdf.to_crs(epsg=31983)
    gdf_projected['x'] = gdf_projected.geometry.x
    gdf_projected['y'] = gdf_projected.geometry.y
    return gdf_projected.drop(columns=['geometry'])

def add_speed_metric(df: pd.DataFrame, group_by: str = 'user_id') -> pd.DataFrame:
    """
    Computes backward-looking speed between consecutive cell changes for each user.
    A constant speed is assigned within each stay/move segment.
    """
    aux_df = df.sort_values([group_by, 'timestamp']).reset_index(drop=True)
    aux_df['timestamp'] = pd.to_datetime(aux_df['timestamp'])
    
    def compute_speed(group: pd.DataFrame) -> pd.DataFrame:
        n = len(group)
        if n == 0:
            return group.assign(speed=np.nan)
        x = group['x'].to_numpy(dtype=float)
        y = group['y'].to_numpy(dtype=float)
        ts = group['timestamp'].to_numpy('datetime64[ns]')
        cell = group['cell_id'].to_numpy()
        pos = np.arange(n)
        # Identify the start index of each new cell segment
        change_mask = np.r_[True, cell[1:] != cell[:-1]]
        change_pos = pos[change_mask]
        # For each index i, find the start of its current segment
        idx_cur = np.searchsorted(change_pos, pos, side='right') - 1
        cur_pos = np.full(n, -1, dtype=int)
        mask_cur = idx_cur >= 0
        cur_pos[mask_cur] = change_pos[idx_cur[mask_cur]]
        # Previous segment start for each row
        prev_idx = idx_cur - 1
        prev_pos = np.full(n, -1, dtype=int)
        mask_prev = prev_idx >= 0
        prev_pos[mask_prev] = change_pos[prev_idx[mask_prev]]
        # Compute speed only when both prev_pos and cur_pos are valid and different
        speed = np.full(n, np.nan, dtype=float)
        valid = (prev_pos != -1) & (cur_pos != -1) & (cur_pos != prev_pos)
        if valid.any():
            dt_h = (ts[cur_pos[valid]] - ts[prev_pos[valid]]) / np.timedelta64(1, 'h')
            dist_km = np.hypot(
                x[cur_pos[valid]] - x[prev_pos[valid]],
                y[cur_pos[valid]] - y[prev_pos[valid]]
            ) / 1000.0
            speed_vals = np.full(valid.sum(), np.nan)
            nonzero = dt_h > 0
            speed_vals[nonzero] = dist_km[nonzero] / dt_h[nonzero]
            speed[valid] = speed_vals
        return group.assign(speed=speed)
    aux_df = aux_df.groupby(group_by, group_keys=False).apply(compute_speed)
    return aux_df

def determine_stay_move_from_speed(df: pd.DataFrame, high_speed_threshold_kmh: float = 10.0, low_speed_threshold_kmh: float = 3.0, group_by: str = 'user_id') -> pd.DataFrame:
    """
    Classify each event as 'move' or 'stay' based on speed thresholds.  
    High threshold marks definite moves; contiguous rows with speed above the low threshold are also marked as moves.  
    """
    def process_user(group: pd.DataFrame) -> pd.DataFrame:
        group = group.sort_values('timestamp').reset_index(drop=True)
        group['speed'] = pd.to_numeric(group['speed'], errors='coerce')
        # Initial classification: speed > high_speed → move
        group['event_type'] = np.where(group['speed'] > high_speed_threshold_kmh, 'move', 'stay')
        seed = group['event_type'].eq('move')
        cand = group['speed'] > low_speed_threshold_kmh
        mask = seed | cand
        # Identify contiguous candidate islands
        block_id = (mask.ne(mask.shift(fill_value=False))).cumsum()
        block_id = block_id.where(mask)
        # If any seed exists in the block, extend moves to include candidates
        has_seed = seed.groupby(block_id).transform('any').fillna(False).astype(bool)
        extend = cand & has_seed
        group['event_type'] = np.where(seed | extend, 'move', 'stay')
        return group
    result = df.groupby(group_by, group_keys=False).apply(process_user)
    return result

def collapse_stay_move(df: pd.DataFrame, time_col: str = 'timestamp', type_col: str = 'event_type', lat_col: str = 'latitude', lon_col: str = 'longitude', group_by: str = 'user_id') -> pd.DataFrame:
    """
    Collapse consecutive events of the same type into segments.  
    For stay segments, compute a time-weighted centroid; for move segments, lat/lon may remain NaN.
    """
    out = df.copy()
    out[time_col] = pd.to_datetime(out[time_col], errors='coerce')
    out = out.sort_values([group_by, time_col])
    out['_seg_id'] = out.groupby(group_by)[type_col].transform(lambda s: (s != s.shift()).cumsum())
    seg_bounds = out.groupby([group_by, '_seg_id']).agg(
        event_type=(type_col, 'first'),
        start_time=(time_col, 'first'),
        end_time=(time_col, 'last'),
    )
    out = out.join(seg_bounds[['end_time']], on=[group_by, '_seg_id'], rsuffix='_seg')
    out['_next_time'] = out.groupby([group_by, '_seg_id'])[time_col].shift(-1)
    w = (out['_next_time'].fillna(out['end_time']) - out[time_col]).dt.total_seconds().clip(lower=0)
    out['_w'] = w
    out['_wlat'] = out['_w'] * out[lat_col]
    out['_wlon'] = out['_w'] * out[lon_col]
    agg = out.groupby([group_by, '_seg_id']).agg(
        event_type=(type_col, 'first'),
        start_time=(time_col, 'first'),
        end_time=(time_col, 'last'),
        w_sum=('_w', 'sum'),
        wlat_sum=('_wlat', 'sum'),
        wlon_sum=('_wlon', 'sum'),
        lat_mean=(lat_col, 'mean'),
        lon_mean=(lon_col, 'mean'),
    )
    agg['duration_s'] = (agg['end_time'] - agg['start_time']).dt.total_seconds()
    stay_mask = agg['event_type'].eq('stay')
    has_weight = agg['w_sum'] > 0
    agg['latitude'] = np.where(stay_mask & has_weight, agg['wlat_sum'] / agg['w_sum'], np.where(stay_mask, agg['lat_mean'], np.nan))
    agg['longitude'] = np.where(stay_mask & has_weight, agg['wlon_sum'] / agg['w_sum'], np.where(stay_mask, agg['lon_mean'], np.nan))
    result = (
        agg.reset_index(level='_seg_id', drop=True).reset_index()[
            [group_by, 'event_type', 'start_time', 'end_time', 'duration_s', 'latitude', 'longitude']
        ].sort_values([group_by, 'start_time']).reset_index(drop=True)
    )
    return result

def map_points_to_admin2(points_df: pd.DataFrame, adm2_gdf: gpd.GeoDataFrame, lat_col: str = 'latitude', lon_col: str = 'longitude') -> pd.DataFrame:
    """
    Assign each point in `points_df` to an Admin2 polygon using a spatial join.  
    Returns a copy of the input DataFrame with added columns `GID_2` and `NAME_2`.  
    """
    gdf = gpd.GeoDataFrame(
        points_df,
        geometry=gpd.points_from_xy(points_df[lon_col], points_df[lat_col]),
        crs='EPSG:4326'
    )
    join = gpd.sjoin(gdf, adm2_gdf[['ID_2', 'NAME_2', 'geometry']], how='left', predicate='within')
    # Return DataFrame without geometry
    return join.drop(columns=['geometry'])


## 4 – Compute OD Flows for Methods A and B
We now compute OD flows using the functions defined above. The overall workflow is:
1. **Filter users whose home is in the target Admin 2 set.**
2. **For each method, compute speeds between consecutive events, classify stay vs move using `high_speed` and `low_speed` thresholds, and collapse segments into trips.**
3. **Assign each origin and destination of a trip to an Admin 2 district via spatial join, then count trips for each OD pair.**

In this exercise we compare two specific methods:
* **Method A** – baseline: `high_speed = 10 km/h`, `low_speed = 3 km/h`.
* **Method B** – alternative: `high_speed = 20 km/h`, `low_speed = 3 km/h`.

After computing the flows for both methods, we merge the tables by origin–destination pair, compute absolute and relative differences, and identify OD pairs where the **absolute relative difference** (|Flow_B – Flow_A| / Flow_A) exceeds a threshold (e.g., 30 %).

In [ ]:
from typing import Tuple, List

def compute_od_flows(events_df: pd.DataFrame,
                     cells_df: pd.DataFrame,
                     target_user_ids: List[int],
                     high_speed: float,
                     low_speed: float) -> pd.DataFrame:
    """
    Compute origin–destination flows at Admin2 level for a given set of users and speed thresholds.
    
    Parameters
    ----------
    events_df : pd.DataFrame
        Raw CDR events with columns: user_id, timestamp, cell_id
    cells_df : pd.DataFrame
        Tower metadata with longitude and latitude
    target_user_ids : list
        IDs of users whose flows should be included
    high_speed : float
        High speed threshold (km/h)
    low_speed : float
        Low speed threshold (km/h)

    Returns
    -------
    pd.DataFrame
        Origin–destination flows with columns: origin_gid, destination_gid, flow
    """
    # Filter events to target users
    target_events = events_df[events_df['user_id'].isin(target_user_ids)].copy()
    if target_events.empty:
        return pd.DataFrame(columns=['origin_gid', 'destination_gid', 'flow'])
    
    # Merge coordinates and project to meters
    ev_proj = add_coordinates_in_meters_to_events(target_events, cells_df)
    
    # Compute speed
    ev_speed = add_speed_metric(ev_proj)
    
    # Classify events into stay/move
    ev_classified = determine_stay_move_from_speed(ev_speed, high_speed_threshold_kmh=high_speed, low_speed_threshold_kmh=low_speed)
    
    # Collapse into segments and only keep stay segments (for origin/destination coordinates)
    segments = collapse_stay_move(ev_classified)
    
    # Remove segments with no duration or no coordinates (could occur if all NaN)
    segments = segments.dropna(subset=['latitude', 'longitude'])
    if segments.empty:
        return pd.DataFrame(columns=['origin_gid', 'destination_gid', 'flow'])
    
    # Map stay segments to admin2 codes
    segments_with_admin = map_points_to_admin2(segments, adm2_gdf)
    
    # Build OD trips: for each user, pair consecutive stays (origin → destination)
    trips = []
    for user, group in segments_with_admin.groupby('user_id'):
        group = group.sort_values('start_time').reset_index(drop=True)
        # iterate over consecutive pairs
        for i in range(len(group) - 1):
            origin_row = group.iloc[i]
            dest_row = group.iloc[i + 1]
            origin_gid = origin_row['ID_2']
            dest_gid = dest_row['ID_2']
            # Only count if both origin and destination codes are valid
            if pd.notna(origin_gid) and pd.notna(dest_gid):
                trips.append((origin_gid, dest_gid))
    # Summarise flows
    if not trips:
        return pd.DataFrame(columns=['origin_gid', 'destination_gid', 'flow'])
    trips_df = pd.DataFrame(trips, columns=['origin_gid', 'destination_gid'])
    flows = trips_df.value_counts().reset_index(name='flow')
    return flows


### Compute OD flows for Method A (10/3 km/h) and Method B (20/3 km/h)
We compute OD flows separately for two sets of speed thresholds. Once flows are available, we merge them on `(origin_gid, destination_gid)`, compute the absolute difference `(Flow_A – Flow_B)`, the signed magnitude `(Flow_B – Flow_A) / Flow_A`, and the percentage change.
We then sort by the percentage change to identify OD pairs that are highly sensitive to the speed threshold (e.g., where |percent_change| > 30 %).

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import json

# Identify user IDs whose home district is in the selected target_admin2 list
target_user_ids = home_join[home_join['ID_2'].isin(target_admin2)]['user_id'].unique().tolist()
print(f'Number of users with home in target Admin2: {len(target_user_ids)}')

# Compute OD flows for Method A
od_A = compute_od_flows(events, cells, target_user_ids, high_speed=10, low_speed=3)
od_A = od_A.rename(columns={'flow': 'flow_A'})
print(f'Method A produced {len(od_A)} OD pairs.')

# Compute OD flows for Method B
od_B = compute_od_flows(events, cells, target_user_ids, high_speed=20, low_speed=3)
od_B = od_B.rename(columns={'flow': 'flow_B'})
print(f'Method B produced {len(od_B)} OD pairs.')

# Merge the two tables on origin and destination
od_merged = pd.merge(od_A, od_B, on=['origin_gid', 'destination_gid'], how='outer').fillna(0)

# Compute absolute and relative differences
# difference (A - B) as requested
od_merged['diff_AB'] = od_merged['flow_A'] - od_merged['flow_B']

# relative diff (B-A)/A  (ratio)  and convert to percent
od_merged['rel_diff'] = np.where(
    od_merged['flow_A'] > 0,
    (od_merged['flow_B'] - od_merged['flow_A']) / od_merged['flow_A'],
    np.nan
)

# magnitude of changes (signed ratio, before x100)
od_merged['magnitude_of_change'] = od_merged['rel_diff']

# percent change
od_merged['pct_change'] = od_merged['rel_diff'] * 100

# Attach Admin2 names instead of GIDs
# adm2_gdf has columns: ID_2, NAME_2, geometry
origin_lookup = (
    adm2_gdf[['ID_2', 'NAME_2']]
    .drop_duplicates()
    .rename(columns={'ID_2': 'origin_gid', 'NAME_2': 'Origin Admin2'})
)

destination_lookup = (
    adm2_gdf[['ID_2', 'NAME_2']]
    .drop_duplicates()
    .rename(columns={'ID_2': 'destination_gid', 'NAME_2': 'Destination Admin2'})
)

od_named = (
    od_merged
    .merge(origin_lookup, on='origin_gid', how='left')
    .merge(destination_lookup, on='destination_gid', how='left')
)

#  keep only inter-district flows (Origin != Destination)
od_named = od_named[od_named['Origin Admin2'] != od_named['Destination Admin2']].copy()

#  keep only OD pairs where BOTH origin and destination are in the specified ID2 list
allowed_ids = [4918, 4877, 4677, 4570, 4688, 4859, 4920]
od_named = od_named[
    od_named['origin_gid'].isin(allowed_ids) &
    od_named['destination_gid'].isin(allowed_ids)
].copy()

# ---------------------------------------------------------
#  Compute population density per Admin2 from shapefile + BRA_adm2_pop.csv
# ---------------------------------------------------------

# 1) project to a metric CRS to compute area (m² → km²)
adm2_area = adm2_gdf.to_crs(epsg=3857).copy()
adm2_area['area_km2'] = adm2_area.geometry.area / 1e6

# 2) join WorldPop totals (adm2_pop must have ID_2 and pop_sum)
adm2_with_pop = adm2_area.merge(
    adm2_pop[['ID_2', 'pop_sum']],
    on='ID_2',
    how='left'
)

# 3) compute population density
adm2_with_pop['pop_density'] = adm2_with_pop['pop_sum'] / adm2_with_pop['area_km2']

# 4) prepare lookup tables for origin and destination density
density_origin = adm2_with_pop[['ID_2', 'pop_density']].rename(
    columns={'ID_2': 'origin_gid', 'pop_density': 'Origin_pop_density'}
)
density_dest = adm2_with_pop[['ID_2', 'pop_density']].rename(
    columns={'ID_2': 'destination_gid', 'pop_density': 'Destination_pop_density'}
)

# 5) merge density into OD table
od_named = (
    od_named
    .merge(density_origin, on='origin_gid', how='left')
    .merge(density_dest, on='destination_gid', how='left')
)

# ---------------------------------------------------------
#  Filter on number of residents (home locations) > 15
# ---------------------------------------------------------

# home_counts has: ID_2, NAME_2, num_subscribers (residents)
res_origin = home_counts[['ID_2', 'num_subscribers']].rename(
    columns={'ID_2': 'origin_gid', 'num_subscribers': 'Origin_residents'}
)
res_dest = home_counts[['ID_2', 'num_subscribers']].rename(
    columns={'ID_2': 'destination_gid', 'num_subscribers': 'Destination_residents'}
)

od_named = (
    od_named
    .merge(res_origin, on='origin_gid', how='left')
    .merge(res_dest, on='destination_gid', how='left')
)

#  keep only OD pairs where BOTH origin and destination have > 15 residents
od_named = od_named[
    (od_named['Origin_residents'] > 15) &
    (od_named['Destination_residents'] > 15)
].copy()

# ---------------------------------------------------------
#  Apply minimum trip_count ≥ 100 (using Method A as baseline)
# ---------------------------------------------------------

od_named = od_named[od_named['flow_A'] >= 100].copy()

# ---------------------------------------------------------
# Sort and export
# ---------------------------------------------------------

# Sort by percent_change (ascending = largest decreases at the top if negative)
od_named = od_named.sort_values(by='pct_change', ascending=True)

# Select and rename columns for output
od_output = od_named[[
    'Origin Admin2',
    'Destination Admin2',
    'flow_A',
    'flow_B',
    'diff_AB',
    'magnitude_of_change',
    'pct_change',
    'Origin_pop_density',
    'Destination_pop_density',
    'Origin_residents',
    'Destination_residents'
]].rename(columns={
    'flow_A': 'Trip count by method A',
    'flow_B': 'Trip count by method B',
    'diff_AB': 'difference (A-B)',
    'magnitude_of_change': 'magnitude_of_change',   # ratio (B-A)/A
    'pct_change': 'percent_change',                 # in %
})

# # Save to CSV
# output_fp = 'od_comparison_admin2_methodA_B_selectedIDs_residents15_flowA100.csv'
# od_output.to_csv(output_fp, index=False)
# print(f'Saved OD comparison table to {output_fp}')

# Show top 20 rows
display(od_output.head(20))



## 5 – Plotting Home Locations for Methods A and B

In addition to comparing the origin–destination flows derived from different speed thresholds, it is often insightful to see where each subscriber’s home is estimated to be under different definitions of ‘day’ and ‘night’.  The **home location** is typically defined as the place where a subscriber spends the most nights and weekends.  Depending on how ‘work hours’ and ‘night‑time’ hours are defined, the candidate home tower can change.

Two alternative sets of hour ranges are considered here:

* **Method A (default)** – work hours are assumed to occur between **05:00 and 22:00**; night‑time is **22:00–05:00**.  These are the same ranges used in the main continuity‑model analysis.
* **Method B (alternative)** – work hours are assumed to occur between **07:00 and 20:00**; night‑time is **20:00–07:00**.  This narrower definition of the working day may shift home/work assignments for some users.

For each user and candidate cell we count:

* the number of **distinct days with at least one event during work hours** (`work_hours_days_cnt`),
* the number of **distinct days with at least one event during the night** (`nighttime_days_cnt`),
* the number of **distinct weekend days** (Saturday/Sunday) (`distinct_weekend_days_count`), and
* the total number of **distinct days with any events** (`distinct_days_count`).

Locations with frequent night‑time and weekend activity are favoured as home.  The `classify_locations` helper below applies the same logic as in the *Meaningful Locations and Usual Environment* notebook but allows us to plug in different hour ranges.  After classification we map the home points for each method on top of the Admin 2 boundaries using Folium’s layer control.  This makes it easy to toggle between Method A and Method B and visually inspect any differences.


In [ ]:

# --- Home location comparison for Methods A and B ---

# Ensure timestamp is datetime
events['timestamp'] = pd.to_datetime(events['timestamp'])

# Define a helper to build a summary table with arbitrary work/night ranges
def build_user_cell_summary(events_df, work_start, work_end, night_start, night_end):
    '''Return aggregated per-user/per-cell metrics using specified hour ranges.'''
    def _distinct_days(dates):
        return dates.dt.date.nunique()

    grouped = events_df.groupby(['user_id', 'cell_id']).agg(
        event_count=('user_id', 'count'),
        distinct_days_count=('timestamp', _distinct_days),
        distinct_weekend_days_count=('timestamp', lambda x: x[x.dt.dayofweek >= 5].dt.date.nunique()),
        work_hours_days_cnt=('timestamp', lambda x: x[(x.dt.hour >= work_start) & (x.dt.hour < work_end)].dt.date.nunique()),
        nighttime_days_cnt=('timestamp', lambda x: x[(x.dt.hour >= night_start) | (x.dt.hour < night_end)].dt.date.nunique())
    ).reset_index()
    return grouped

# Classification helper adapted from the Meaningful Locations notebook
def classify_locations(df,
                       top_n_locations=4,
                       home_min_night_days=14,
                       home_min_weekend_days=2,
                       work_min_work_hours_days=10,
                       work_max_weekend_days=2,
                       min_distinct_days=5,
                       other_location_min_days=5):
    '''Classifies locations as 'home', 'work', or 'other' based on tunable parameters.\n    This function is identical to the one used in the Meaningful Locations notebook.'''
    df_classified = df.copy()
    # Rank locations per user by total distinct days
    df_classified['rank'] = df_classified.groupby('user_id')['distinct_days_count'].rank(method='first', ascending=False)
    is_top_location = df_classified['rank'] <= top_n_locations
    is_significant_enough = df_classified['distinct_days_count'] >= min_distinct_days
    is_home = (
        (df_classified['nighttime_days_cnt'] >= home_min_night_days) &
        (df_classified['distinct_weekend_days_count'] >= home_min_weekend_days)
    )
    is_work = (
        (df_classified['work_hours_days_cnt'] >= work_min_work_hours_days) &
        (df_classified['distinct_weekend_days_count'] <= work_max_weekend_days)
    )
    is_other = (df_classified['distinct_days_count'] >= other_location_min_days)
    conditions = [is_top_location & is_significant_enough & is_home,
                  is_top_location & is_significant_enough & is_work,
                  is_other]
    choices = ['home', 'work', 'other']
    df_classified['location_type'] = np.select(conditions, choices, default='NaN')
    return df_classified

# Build per-user summaries for Method A and Method B
summary_A = build_user_cell_summary(events, work_start=5, work_end=22, night_start=22, night_end=5)
summary_B = build_user_cell_summary(events, work_start=7, work_end=20, night_start=20, night_end=7)

# Classify locations
classified_A = classify_locations(summary_A)
classified_B = classify_locations(summary_B)

# Extract home locations for each user (if multiple candidates exist, take the one with most night days)
homes_A = (
    classified_A[classified_A['location_type'] == 'home']
    .sort_values(['user_id', 'nighttime_days_cnt', 'distinct_weekend_days_count'], ascending=[True, False, False])
    .drop_duplicates(subset=['user_id'])
)
homes_B = (
    classified_B[classified_B['location_type'] == 'home']
    .sort_values(['user_id', 'nighttime_days_cnt', 'distinct_weekend_days_count'], ascending=[True, False, False])
    .drop_duplicates(subset=['user_id'])
)

# Merge with cells metadata to obtain coordinates
homes_A = homes_A.merge(cells[['cell_id', 'longitude', 'latitude']], on='cell_id', how='left')
homes_B = homes_B.merge(cells[['cell_id', 'longitude', 'latitude']], on='cell_id', how='left')

# Convert to GeoDataFrame for mapping
homes_A_gdf = gpd.GeoDataFrame(homes_A, geometry=gpd.points_from_xy(homes_A['longitude'], homes_A['latitude']), crs='EPSG:4326')
homes_B_gdf = gpd.GeoDataFrame(homes_B, geometry=gpd.points_from_xy(homes_B['longitude'], homes_B['latitude']), crs='EPSG:4326')

# Base map centred on mean coordinate of all homes
if not homes_A_gdf.empty:
    centre_lat = float(homes_A_gdf['latitude'].mean())
    centre_lon = float(homes_A_gdf['longitude'].mean())
else:
    centre_lat, centre_lon = 0.0, 0.0

m = folium.Map(location=[centre_lat, centre_lon], zoom_start=6)

# Feature group for Method A homes
fg_A = folium.FeatureGroup(name='Home (Method A)')
for _, row in homes_A_gdf.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"User {row['user_id']} – Cell {row['cell_id']}"
    ).add_to(fg_A)

# Feature group for Method B homes
fg_B = folium.FeatureGroup(name='Home (Method B)')
for _, row in homes_B_gdf.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f"User {row['user_id']} – Cell {row['cell_id']}"
    ).add_to(fg_B)

# Add layers and control
fg_A.add_to(m)
fg_B.add_to(m)
folium.LayerControl().add_to(m)

# Display map in notebook
m

# Optionally save to HTML
# m.save("home_location_comparison_methodA_B.html")